# kerasに慣れよう1

入力ベクトル$(x_{1},x_{2},x_{3},x_{4},x_{5})$ ($x_i \in [0,1]$)の各要素の加算結果が2.5以上で1,未満で0を出すモデル  
$f(x) = if\ \sum^5_{i=1}x_i \ \geq\ 2.5 \ then\  1\  else\  0$  
参考:[無から始めるKeras 第1回][1]  
[1]:https://qiita.com/Ishotihadus/items/c2f864c0cde3d17b7efb

## ライブラリなどの準備

In [1]:
#ライブラリ
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation

#GPUの仕様に関する設定
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto(
    gpu_options = tf.GPUOptions(
        visible_device_list = "0",
        allow_growth = True,
        per_process_gpu_memory_fraction = 0.1))
sess = tf.Session(config=config)
K.set_session(sess)

Using TensorFlow backend.


## データセットの生成

In [2]:
#学習データセット
data = np.random.rand(10000,5) #1000個の5次元ベクトル
labels = (np.sum(data, axis=1) > 2.5) * 1 #ラベル(0,1)
labels = np_utils.to_categorical(labels)  #ラベル(onehot)
#onehotラベルはこんな感じ
#学習データ
print("学習データ")
print(data[:3])
print("正解(onehot)")
print(labels[:3])

学習データ
[[0.68436375 0.25782697 0.19639783 0.26483645 0.26701841]
 [0.2174457  0.07857552 0.97858756 0.93750869 0.78997014]
 [0.58724019 0.97682438 0.62271724 0.37918633 0.93716185]]
正解(onehot)
[[1. 0.]
 [0. 1.]
 [0. 1.]]


## ネットワークの構築

kerasでは層を積み上げていくイメージ  
model.add以外にもこんな書き方もできる
```
model = Sequential([
    Dense(20, input_dim=5, activation='relu'),
    Dense(2,activation='softmax')
])
```
Functional APIを使った記法  
複雑なモデルを作るときに使用，最初はこんなのあるんだ見たいな気持ちでOK
```
from keras.layers import Input, Dense, Activation
from keras.models import Model
inputs = Input(shape=(5,))
x = Dense(20)(inputs)
x = Activation('relu')(x)
outputs = Dense(2,activation='softmax')(x)
model = Model(inputs=inputs, outputs = outputs)
```
### 活性化関数
- sigmoid
- relu
- tanh  

### バッチ正規化層
ミニバッチ内での直前の出力を正規化する
- 学習を早く進行できる
- 初期値に依存しない
- 過学習抑制
```
from keras.layers.normalization import BatchNormalization
```

In [3]:
#Sequential:層を積み上げる単純なモデル
model = Sequential()
model.add(Dense(20, input_dim=5))
model.add(Activation('relu'))
model.add(Dense(2, activation='softmax'))

#ネットワーク構造の出力
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                120       
_________________________________________________________________
activation_1 (Activation)    (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 42        
Total params: 162
Trainable params: 162
Non-trainable params: 0
_________________________________________________________________


## 学習

### 最適化関数(optimizer)  
- SGD(確率的勾配降下法):  
傾きに比例して移動する  
- Momentam:  
ボールが転がるイメージ  
- AdaGrad:  
学習係数を下げていく(最初は大きく動かし，終盤は微調整)
- Adam:  
MomentumとAdaGradの融合    

例(optimizerを設定する場合
```
from keras import optimizers
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999,epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer = adam, 'categorical_crossentropy', metrics=['accuracy'])
```
### 損失関数(loss)
- mean_squared_errer:二乗平均誤差
- mean_absolute_error:絶対平均誤差
- hinge:マイナスのところは0,プラスのところはその値
- categorical_crossentropy:複数クラスの交差エントロピー

In [4]:
#compli:最適化関数,損失関数,評価指標
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
#学習
#fit;入力，出力，バッチサイズ(一度の更新に使用するデータ数), エポック数(学習回数),検証セットの割合(ランダムではないので注意)
model.fit(data, labels, batch_size = 100, epochs=150, validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/150
8000/8000 [==============================] - 1s 80us/step - loss: 0.7576 - acc: 0.4397 - val_loss: 0.7030 - val_acc: 0.4830
Epoch 2/150
8000/8000 [==============================] - 0s 45us/step - loss: 0.6772 - acc: 0.5434 - val_loss: 0.6493 - val_acc: 0.5775
Epoch 3/150
8000/8000 [==============================] - 0s 44us/step - loss: 0.6158 - acc: 0.6624 - val_loss: 0.5795 - val_acc: 0.7315
Epoch 4/150
8000/8000 [==============================] - 0s 43us/step - loss: 0.5385 - acc: 0.7990 - val_loss: 0.4975 - val_acc: 0.8200
Epoch 5/150
8000/8000 [==============================] - 0s 44us/step - loss: 0.4549 - acc: 0.8844 - val_loss: 0.4160 - val_acc: 0.8930
Epoch 6/150
8000/8000 [==============================] - 0s 43us/step - loss: 0.3770 - acc: 0.9390 - val_loss: 0.3454 - val_acc: 0.9525
Epoch 7/150
8000/8000 [==============================] - 0s 42us/step - loss: 0.3140 - acc: 0.9650 - val_loss: 0.2913 - val_acc: 0.9590


8000/8000 [==============================] - 0s 43us/step - loss: 0.0480 - acc: 0.9951 - val_loss: 0.0499 - val_acc: 0.9910
Epoch 61/150
8000/8000 [==============================] - 0s 44us/step - loss: 0.0477 - acc: 0.9939 - val_loss: 0.0498 - val_acc: 0.9900
Epoch 62/150
8000/8000 [==============================] - 0s 44us/step - loss: 0.0468 - acc: 0.9951 - val_loss: 0.0479 - val_acc: 0.9940
Epoch 63/150
8000/8000 [==============================] - 0s 43us/step - loss: 0.0458 - acc: 0.9954 - val_loss: 0.0481 - val_acc: 0.9920
Epoch 64/150
8000/8000 [==============================] - 0s 43us/step - loss: 0.0456 - acc: 0.9946 - val_loss: 0.0471 - val_acc: 0.9950
Epoch 65/150
8000/8000 [==============================] - 0s 45us/step - loss: 0.0453 - acc: 0.9950 - val_loss: 0.0462 - val_acc: 0.9970
Epoch 66/150
8000/8000 [==============================] - 0s 46us/step - loss: 0.0445 - acc: 0.9951 - val_loss: 0.0458 - val_acc: 0.9960
Epoch 67/150
8000/8000 [==============================

Epoch 120/150
8000/8000 [==============================] - 0s 43us/step - loss: 0.0306 - acc: 0.9970 - val_loss: 0.0315 - val_acc: 0.9965
Epoch 121/150
8000/8000 [==============================] - 0s 43us/step - loss: 0.0306 - acc: 0.9960 - val_loss: 0.0352 - val_acc: 0.9880
Epoch 122/150
8000/8000 [==============================] - 0s 44us/step - loss: 0.0307 - acc: 0.9948 - val_loss: 0.0313 - val_acc: 0.9970
Epoch 123/150
8000/8000 [==============================] - 0s 44us/step - loss: 0.0303 - acc: 0.9959 - val_loss: 0.0322 - val_acc: 0.9950
Epoch 124/150
8000/8000 [==============================] - 0s 43us/step - loss: 0.0299 - acc: 0.9964 - val_loss: 0.0308 - val_acc: 0.9970
Epoch 125/150
8000/8000 [==============================] - 0s 43us/step - loss: 0.0299 - acc: 0.9965 - val_loss: 0.0305 - val_acc: 0.9970
Epoch 126/150
8000/8000 [==============================] - 0s 44us/step - loss: 0.0300 - acc: 0.9959 - val_loss: 0.0307 - val_acc: 0.9950
Epoch 127/150
8000/8000 [=========

## 未知データの予測

In [5]:
#評価データセット
test_data = np.random.rand(1000,5)
test_label = (np.sum(test_data, axis=1) > 2.5) * 1 

In [6]:
print("入力")
print(test_data[0])
print("正解")
print(test_label[0])
print("予測結果の確率(モデルの出力)")
#出力結果の確率
print(model.predict(test_data[0:1]))
print("確率が最大の次元")
#最も大きい次元を出力とする
print(np.argmax(model.predict(test_data[0:1])))

入力
[0.75161398 0.43540378 0.80316324 0.78189133 0.74710614]
正解
1
予測結果の確率(モデルの出力)
[[1.02109676e-16 1.00000000e+00]]
確率が最大の次元
1


In [7]:
predict = np.argmax(model.predict(test_data), axis=1)
#predictの中身
print("各データに対する予測")
print(predict[:3])
print("正解と予測の比較")
print((predict==test_label)[:3])
#sumを取るとtrueの数を集計する
print("認識率")
print(sum(predict == test_label) /1000)

各データに対する予測
[1 1 1]
正解と予測の比較
[ True  True  True]
認識率
0.994


## モデルの中身(重みとバイアスを取得)

In [8]:
weight = model.get_weights()
print("hidden weight")
print(weight[0]) #隠れ層重み
print("hidden bias")
print(weight[1]) #隠れ層バイアス
print("output weight")
print(weight[2]) #出力層重み
print("output bias")
print(weight[3]) #出力層バイアス

hidden weight
[[ 0.41390866 -0.29119664  0.5892099  -0.3272763  -0.46100035  0.33359239
  -0.3317244   1.0288044   0.49827272 -0.17400582  1.003807   -0.86153907
  -0.2988587   0.68076104 -0.66668355  0.6679993  -0.583837   -0.4433174
  -0.6278939   0.07720908]
 [ 0.50179195 -0.83415407  0.6929002   0.0748447   0.13634607  0.8414961
  -0.88150716  1.0583111   0.659134   -0.02062757  0.51028615  0.05115708
  -0.45406058  0.43187776 -0.4353062   0.77067643 -0.5279241  -0.21292113
  -0.13896658 -0.27174053]
 [ 1.131728   -0.24077836  0.38941157 -1.1562248  -0.28829515  1.1540117
  -0.40588653  0.45535478  0.85254616 -0.76199406  0.8861617  -0.03433726
  -0.46503213  1.0317872  -0.09902468  0.71349865  0.15029283 -0.50975573
  -0.20205204 -0.37150443]
 [ 0.6384488  -0.5572955   0.8529426  -0.05861638 -0.2512384   0.84063435
   0.0245974   0.19556046  0.64932925  0.10105489  0.6166557  -0.10691435
  -0.7753219   0.5789025  -0.733306    0.80170023 -0.4559402  -0.03529052
  -0.6220595  -0.133